In [9]:
import cv2
import numpy as np
import math

#Identifica as bordas
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    canny = cv2.Canny(blur, 200,400)
    return canny

def steering_correction(central_line):
    delta = 90-abs(math.atan(central_line[0])*(180/np.pi))
    if central_line[0] < 0:
        print("Turn", delta, "degrees left" )
    else:
        print("Turn", delta, "degrees right")
        
def show_img(image, central_line):
    cv2.imshow("teste", image)
    steering_correction(central_line)

def region_of_interest(edges):
    height, width = edges.shape
    poly = np.array([[(0,height), (0,height*(1/2)), (width, height*(1/2)), (width, height)]], np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, poly, 255)
    cropped_edges = cv2.bitwise_and(mask, edges)
    return cropped_edges

def find_lines(cropped):
    rho = 1
    angle = np.pi/180
    min_threshold = 50
    line_segments = cv2.HoughLinesP(
        cropped,
        rho,
        angle,
        min_threshold,
        np.array([]),
        minLineLength=8,
        maxLineGap=4
    )
    
    return line_segments

def make_coordinates(image, lines):
    if lines.shape[0] == 0:
        return
    slope, intercept = lines
    y1 = image.shape[0]
    y2 = int(y1*(1/2))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return [x1,y1,x2,y2] 

def average_slope_intercept(image, lines):
    
    left_lines = []
    right_lines = []
    
    x_left_boundary = (1/3)*image.shape[1]
    x_right_boundary = (2/3)*image.shape[1] 
    
    if lines is None:
        print("The robot cannot recognize anything!")
        return None
    
    for i in range(lines.shape[0]):
        x1,y1,x2,y2 = lines[i][0]
        fit = np.polyfit((x1,x2),(y1,y2), 1)
        if fit[0]>0:
            if fit[1] < x_left_boundary:
                right_lines.append(fit)
        if fit[0]<0:
            if fit[1] > x_right_boundary:
                left_lines.append(fit)
            
    if len(left_lines)>0:       
        left_average = np.average(left_lines, axis = 0)
    else:
        left_average = np.array([])
        
    if len(right_lines)>0:
        right_average = np.average(right_lines, axis = 0)
    else:
        right_average = np.array([])
        
    return [make_coordinates(image, left_average), make_coordinates(image, right_average)]

def make_central_line(image, lines):
    if lines[0] is None:
        middle_line = lines[1]
        x1,y1,x2,y2 = middle_line
        fit = np.polyfit((x1,x2),(y1,y2), 1)
        return [make_coordinates(image,fit), fit]
    if lines[1] is None:
        middle_line = lines[0]
        x1,y1,x2,y2 = middle_line
        fit = np.polyfit((x1,x2),(y1,y2), 1)
        return [make_coordinates(image,fit), fit]
    
    middle_line = np.average(lines, axis = 0)
    x1,y1,x2,y2 = middle_line
    fit = np.polyfit((x1,x2),(y1,y2), 1)
    return [make_coordinates(image,fit), fit]

def display_lines(image, lines, central_line):
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            x1,y1,x2,y2 = line
            print(line)
            cv2.line(line_image, (x1,y1),(x2,y2),(0,0,255),5)
    x1,y1,x2,y2 = central_line
    cv2.line(line_image, (x1,y1),(x2,y2),(255,0,0),5)
    return line_image

def image_vision(img):
    ret, image = img.read()
    edges = canny(image)
    cropped_edges = region_of_interest(edges)
    lines = average_slope_intercept(image, find_lines(cropped_edges))
    if lines is not None:
        central_line = make_central_line(image, lines)
        line_image = display_lines(image, lines, central_line[0])
        combo_image = cv2.addWeighted(line_image, 0.8, image, 1, 1)
        show_img(combo_image, central_line[1])
        
def video_vision(video_name):
    image = cv2.VideoCapture(video_name)
    while (image.isOpened()):
        image_vision(image)
        if (cv2.waitKey(25) & 0xFF) == 27:
            cv2.destroyAllWindows()
            break
    image.release()

In [14]:
video_vision('visao_02.mp4')

[8, 480, 56, 240]
[283, 480, 229, 240]
Turn 0.7161599454704088 degrees right
[12, 480, 49, 240]
[283, 480, 229, 240]
Turn 2.0283777159584417 degrees right
[4, 480, 74, 240]
[283, 480, 229, 240]
Turn 1.909152432996379 degrees left
[15, 480, 57, 240]
[282, 480, 229, 240]
Turn 1.312798496975006 degrees right
[3, 480, 74, 240]
[282, 480, 229, 240]
Turn 2.1475854282984983 degrees left
[1, 480, 73, 240]
[282, 480, 229, 240]
Turn 2.266774542094467 degrees left
[4, 480, 73, 240]
[285, 480, 230, 240]
Turn 1.6706532713999422 degrees left
[12, 480, 58, 240]
[282, 480, 229, 240]
Turn 0.8355042246910074 degrees right
[8, 480, 60, 240]
[284, 480, 230, 240]
Turn 0.2387310330989294 degrees right
[3, 480, 71, 240]
[280, 480, 226, 240]
Turn 1.6706532713999707 degrees left
[11, 480, 46, 240]
[282, 480, 228, 240]
Turn 2.266774542094467 degrees right
[-3, 480, 85, 240]
[282, 480, 228, 240]
Turn 4.051683837828435 degrees left
[-8, 480, 99, 240]
[279, 480, 227, 240]
Turn 6.53663364879705 degrees left
[13, 48

<ipython-input-9-daec28c32bae>:125: RankWarning: Polyfit may be poorly conditioned
  central_line = make_central_line(image, lines)


[-4, 480, 82, 240]
[282, 480, 230, 240]
Turn 4.051683837828449 degrees left
[-3, 480, 87, 240]
[285, 480, 231, 240]
Turn 4.289153328818998 degrees left
[7, 480, 59, 240]
[282, 480, 229, 240]
Turn 0.11936603462520168 degrees right
[11, 480, 47, 240]
[282, 480, 229, 240]
Turn 2.0283777159584417 degrees right
[6, 480, 62, 240]
[284, 480, 230, 240]
Turn 0.23873103309888677 degrees left
[5, 480, 59, 240]
[281, 480, 228, 240]
Turn 0.11936603462520168 degrees left


<ipython-input-9-daec28c32bae>:125: RankWarning: Polyfit may be poorly conditioned
  central_line = make_central_line(image, lines)


[5, 480, 58, 240]
[281, 480, 228, 240]
Turn 38.46519018629743 degrees right
[8, 480, 54, 240]
[280, 480, 228, 240]
Turn 0.7161599454703946 degrees right
[4, 480, 57, 240]
[280, 480, 229, 240]
Turn 0.23873103309890098 degrees left
[11, 480, 52, 240]
[278, 480, 230, 240]
Turn 0.835504224690979 degrees right
[2, 480, 53, 240]
[280, 480, 231, 240]
Turn 0.23873103309890098 degrees left
[6, 480, 45, 240]
[280, 480, 232, 240]
Turn 1.074169998372625 degrees right
[-3, 480, 74, 240]
[279, 480, 231, 240]
Turn 3.4574173746236028 degrees left
[-9, 480, 88, 240]
[281, 480, 233, 240]
Turn 5.828752960735201 degrees left
[7, 480, 46, 240]
[282, 480, 234, 240]
Turn 1.074169998372625 degrees right
[-8, 480, 91, 240]
[280, 480, 233, 240]
Turn 6.182930165948264 degrees left
[-6, 480, 91, 240]
[281, 480, 234, 240]
Turn 5.946863053973502 degrees left
[-3, 480, 76, 240]
[280, 480, 234, 240]
Turn 3.9328962722305505 degrees left
[0, 480, 67, 240]
[283, 480, 235, 240]
Turn 2.266774542094467 degrees left
[1, 480

...

[240, 480, 252, 240]
Turn 2.8624052261117043 degrees left
[219, 480, 239, 240]
Turn 4.7636416907261605 degrees left
[248, 480, 258, 240]
Turn 2.3859440303888135 degrees left
[223, 480, 241, 240]
Turn 4.289153328819026 degrees left
[231, 480, 248, 240]
Turn 4.051683837828421 degrees left
[222, 480, 240, 240]
Turn 4.289153328819012 degrees left
[218, 480, 235, 240]
Turn 4.051683837828463 degrees left


<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = a

[217, 480, 233, 240]
Turn 3.8140748342903663 degrees left
[250, 480, 257, 240]
Turn 1.6706532713999565 degrees left
[229, 480, 242, 240]
Turn 3.1004914498077767 degrees left
[242, 480, 252, 240]
Turn 2.3859440303887993 degrees left
[225, 480, 241, 240]
Turn 3.8140748342903805 degrees left
[225, 480, 241, 240]
Turn 3.8140748342903805 degrees left
[225, 480, 241, 240]
Turn 3.8140748342903805 degrees left
[248, 480, 257, 240]
Turn 2.1475854282984983 degrees left
[215, 480, 238, 240]
Turn 5.4741282006091865 degrees left


<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = average_slope_intercept(image, find_lines(cropped_edges))
<ipython-input-9-daec28c32bae>:123: RankWarning: Polyfit may be poorly conditioned
  lines = a

error: OpenCV(4.5.2) /tmp/pip-req-build-eirhwqtr/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
